In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("../../../datasets/ubertext_gec/pre_post_processing_ubertext_gec.csv")
df

In [ ]:
import re

PATTERN = (
    r'(?i)'  # нечутливість до регістру
    r'(👉|'                                 # 1. емодзі "👉"
    r'підписатися|'                         # 2. "підписатися"
    r'підписуватися|'                       # 3. "підписуватися"
    r'підписуйтеся\s+на\s+укрінформ\.?|'    # 4. "Підписуйтеся на Укрінформ."
    r'підписуйтеся\s+на\s+zn\.ua\.?|'       # 5. "Підписуйтеся на ZN.UA."
    r'підписатися\s+на\s+zn\.ua\.?|'        # **НОВИЙ**: "Підписатися на ZN.UA."
    r'приєднуйтесь\s+до\s+zn\.ua\.?|'       # **НОВИЙ**: "Приєднуйтесь до ZN.UA."
    r'підпишіться\s+на\s+нв|'               # 6. "Підпишіться на НВ"
    r'підтримати\s+нв|'                     # 7. "Підтримати НВ"
    r'апостроф\s*\|\s*підписатися|'         # 8. "апостроф | підписатися"
    r'новина\s*російською|'                 # 9. "новина російською"
    r'новостьна\s*русском|'                 # 10. "новостьна русском"
    r'(?:[A-Za-z]+|[^\s]*@[^\s]*)\s*$|'                           # 11. "@"
    r'(?<![.\w])ua(?![.\w])|'               # 12. "ua" як окреме слово (не в доменах)
    r'facebook\s*\|\s*instagram\s*\|\s*twitter|'  # 13. "Facebook | Instagram | Twitter"
    r'facebook\s*\|\s*youtube\s*\|\s*instagram\s*\|\s*telegram|'  # 14. ще один перелік соцмереж
    r'📚\s*(?:этот материал также можно читать(?:\s*на\s*русском)?|'
    r'цей матеріал також доступний російською мовою)|'  # 15. матеріали з емодзі "📚"
    r'статтю\s+можна\s+прочитати\s*російською\s+мовою|'  # 16. "Статтю можна прочитати російською мовою"
    r'(?:підписат(?:ись|атися)\s*\|\s*сайт\s*\|\s*(?:fb|facebook)\s*\|\s*youtube\s*\|\s*підтримати\s*нас)|'  # 17. блок "Підписатись | Сайт | FB | ..."
    r'статья\s+также\s+доступна\s*русском|'             # 18. "Статья также доступна русском"
    r'стаття\s+також\s+доступна\s+російською|'          # 19. "Стаття також доступна російською"
    r'📷\s*юрій\s+журавель|'                             # 20. "📷 Юрій Журавель"
    # Футерні кейси для Espreso:
    r'▪️\s*більше\s+новин\s+читайте\s+на\s+espreso(?:\s*tv\.?)?(?:\s*.*)?|'
    r'tv\.|'
    r'еспресо\s*\||'
    # Кейси для читання російською:
    r'читать\s*нарусск\s*ом\.?|'             # "Читать нарусск ом."
    r'читайте\s+російською\.?|'              # "Читайте російською."
    # Промо-рядок «Усі новини світу ТУТ ➡»
    r'усі\s+новини\s+світу\s+тут\s*➡?\s*https?://\S+|'
    # "🔷️..."
    r'🔷️\s*підписатися\s+на\s+zn\.?'
    r').*?$'
)


def remove_footer_lines(text):
    pattern = PATTERN

    cleaned = re.sub(pattern, '', text, flags=re.MULTILINE)
    cleaned = re.sub(r'\n+', '\n', cleaned).strip()

    return cleaned

df['feature_cleaned'] = df['feature'].apply(remove_footer_lines)
df['target_cleaned'] = df['target'].apply(remove_footer_lines)

In [ ]:
for sample_index in df.sample(15).index:
    sample = df.iloc[sample_index]

    print("> Feature: ")
    print(sample["feature"])

    print("> Cleaned Feature: ")
    print(sample["feature_cleaned"])

    print("> Target")
    print(sample["target"])

    print("> Cleaned Target: ")
    print(sample["target_cleaned"])

    print("------------------------------------------")

In [ ]:
df["feature_len"] = df.feature.str.len()
df["feature_cleaned_len"] = df.feature_cleaned.str.len()
df["target_len"] = df.target.str.len()
df["target_cleaned_len"] = df.target_cleaned.str.len()

df["feature_diff"] = df["feature_len"] - df["feature_cleaned_len"]
df["target_diff"] = df["target_len"] - df["target_cleaned_len"]
df

In [ ]:
import holoviews as hv
import hvplot.pandas

(df.target_diff.hvplot.box(width=1000, grid=True, invert=True, tools=["zoom_in", "zoom_out", "hover"], label="Target") + df.feature_diff.hvplot.box(width=1000, grid=True, invert=True, tools=["zoom_in", "zoom_out", "hover"], label="Feature")).cols(1)

In [ ]:
df["target_final"] = df.apply(lambda x: x["target"] if x["target_diff"] > 30 else x["target_cleaned"], axis=1)
df["feature_final"] = df.apply(lambda x: x["feature"] if x["feature_diff"] > 60 else x["feature_cleaned"], axis=1)

df

In [ ]:
len(df[df["target_diff"] > 30]), len(df[df["feature_diff"] > 60])

In [ ]:
df.loc[:, [
    "index",
    "feature_final",
    "target_final"
          ]].rename(columns={
    "feature_final": "feature",
    "target_final": "target"
}).to_csv("../../../datasets/ubertext_gec/ubertext_gec.csv", index=False)

In [ ]:
post_df = pd.read_csv("../../../datasets/ubertext_gec/ubertext_gec.csv")

In [ ]:
for sample_index in df.sample(5).index:
    sample = post_df.iloc[sample_index]

    print(sample_index)

    print("> Feature: ")
    print(sample["feature"])

    print("> Target")
    print(sample["target"])

    print("------------------------------------------")